In [ ]:
import json
from datetime import datetime

from Crypto.Hash import keccak

from random import randint
from math import floor, ceil

Step 0: The Blockchain

In [ ]:
class Block:

    #blockheaders
    def __init__(self, block_data):
        self.block_data = block_data

    # calculate the hash of the block
    @staticmethod
    def keccackHash(dictionary):
        byte_rep = str.encode(json.dumps(dictionary, sort_keys=True, default=str)) 
        k = keccak.new(digest_bits=256)
        k.update(byte_rep)
        return k.hexdigest()
    
    # calculate the target hash based on the last block's difficulty
    @staticmethod
    def calc_target_hash(last_block):
        HASH_LENGTH = 64
        MAX_HASH_VALUE = int('f' * HASH_LENGTH, 16)

        value =  hex(MAX_HASH_VALUE // last_block.block_data['difficulty'])[2:]

        return '0' * (HASH_LENGTH - len(value)) + value

    # adjust difficulty based on the last block's timestamp and difficulty
    @staticmethod
    def adjust_difficulty(last_block, current_datetime):
        
        last_difficulty = last_block.block_data['difficulty']

        timestamp = int(current_datetime.timestamp() * 1000)
        last_timestamp = int(last_block.block_data['timestamp'].timestamp() * 1000)

        MINE_RATE = 10 * 1000 # 13 seconds

        # if the block was mined too quickly, increase the difficulty
        if timestamp - last_timestamp < MINE_RATE:
            difficulty = last_difficulty + 1
        # if the block was mined too slowly, decrease the difficulty
        else:
            difficulty = last_difficulty - 1

        if difficulty < 1:
            difficulty = 1

        return difficulty

    # mine a block
    @staticmethod
    def mine_block(last_block, beneficiary):
        target = Block.calc_target_hash(last_block)
         
        # mine until the hash is under the target
        while True:
            truncated_block_headers = {
                'parent_hash': Block.keccackHash(last_block.block_data),
                'beneficiary': beneficiary,
                'difficulty': Block.adjust_difficulty(last_block, datetime.now()),
                'number': last_block.block_data['number'] + 1,
                'timestamp': datetime.now()
            }

            header = Block.keccackHash(truncated_block_headers)
            nonce = randint(0, 2 ** 64)

            k = keccak.new(digest_bits=256) 
            k.update(
                str.encode(str(int(header, 16) + nonce))
            )
            under_target_hash = k.hexdigest()
            
            # if the hash is under the target, we have mined a block
            if under_target_hash < target:
                break

        print(f'under_target_hash: {under_target_hash}')
        print(f'target: {target}')
        block_headers = truncated_block_headers
        block_headers['nonce'] = nonce

        return Block(block_headers)
    
    # validate a block
    @staticmethod
    def validateBlock(last_block, block):

        # check if the parent hash is correct
        if Block.keccackHash(last_block.block_data) != block.block_data['parent_hash']:
            print('parent hash is not correct')
            return False
        
        # check if the block number is correct 
        if last_block.block_data['number'] + 1 != block.block_data['number']:
            print('block number is not correct')
            return False
        
        # check if the timestamp is correct
        if  abs(last_block.block_data['difficulty'] - block.block_data['difficulty']) > 1:
            print('difficulty is not correct')
            return False  

        # check if proof of work is correct 
        target = Block.calc_target_hash(last_block)

        truncated_block_headers = {
                'parent_hash': block.block_data['parent_hash'],
                'beneficiary': block.block_data['beneficiary'],
                'difficulty': block.block_data['difficulty'],
                'number': block.block_data['number'],
                'timestamp': block.block_data['timestamp']
            }
        nonce = block.block_data['nonce']

        header = Block.keccackHash(truncated_block_headers)

        k = keccak.new(digest_bits=256) 
        k.update(
            str.encode(str(int(header, 16) + nonce))
        )
        under_target_hash = k.hexdigest()
            
        if under_target_hash > target:
            print('hash is not correct')
            return False
        

        return True   

In [ ]:
GENESIS_BLOCK = Block({
    'parentHash'  : hex((randint(0, 2 ** 64)))[2:],
    'beneficiary' : 'the-first-reciever', 
    'difficulty'  : 100000,
    'number'      : 0,
    'timestamp'   : datetime.now(),
    'nonce'       : 0
})

In [ ]:
class Blockchain:
    def __init__(self):

        self.chain = [GENESIS_BLOCK]
        
        
        #self.all_transactions = []
        #self.genesis_block()

In [ ]:
blockchain = Blockchain()
print(blockchain.chain[-1].block_data)

In [ ]:
for i in range(1000):

    print()

    
    new_block = Block.mine_block(
        last_block= blockchain.chain[-1],
        beneficiary= 'beneficiary'
    )
    
    if Block.validateBlock(blockchain.chain[-1], new_block):
        blockchain.chain.append(new_block)
    else:
        print('invalid block')
        break

    for key, item in new_block.block_data.items():
        print(key, item)



Step 1: The Network